<a href="https://colab.research.google.com/github/Muhammadsulton1/ML/blob/main/ViT_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tarfile
zipfile = '/content/drive/MyDrive/autoriaNumberplateOcrRu-2021-09-01.tar.gz'
if zipfile.endswith("tar.gz"):
    tar = tarfile.open(zipfile, "r:gz")
elif zipfile.endswith("tar"):
    tar = tarfile.open(zipfile, "r:")
tar.extractall()
tar.close()

In [ ]:
import json
import os
import matplotlib.pyplot as plt
import torchvision
import albumentations as A
import time

In [ ]:
def json_to_txt(path_name_to_save,path):
  for name in os.listdir(path):
    with open(os.path.join(path, name), 'r') as f:
      data = json.loads(f.read())
      img_name = data.get('name') + '.png'
      description = data.get('description')
      with open(path_name_to_save + ".txt", "a") as text_file:
        text_file.write('{},{}'.format(img_name, description + '\n'))
        text_file.close()

In [ ]:
json_to_txt('train_label','OCR_Licplate/train/ann')

In [ ]:
json_to_txt('valid_label','OCR_Licplate/val/ann')

In [ ]:
json_to_txt('test_label','OCR_Licplate/test/ann')

In [ ]:
def statistics_label_cnt(lbl_path, lbl_cnt_map):
    with open(lbl_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split(',')
            img_name = items[0]
            lbl_str = items[1].strip()
            for lbl in lbl_str:
                if lbl not in lbl_cnt_map.keys():
                    lbl_cnt_map[lbl] = 1
                else:
                    lbl_cnt_map[lbl] += 1


def statistics_max_len_label(lbl_path):
    max_len = -1
    with open(lbl_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split(',')
            img_name = items[0]
            lbl_str = items[1].strip()
            lbl_len = len(lbl_str)
            max_len = max_len if max_len > lbl_len else lbl_len
    return max_len


def load_lbl2id_map(lbl2id_map_path):
    lbl2id_map = dict()
    id2lbl_map = dict()
    with open(lbl2id_map_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split(' ')
            label = items[0]
            cur_id = int(items[1])
            lbl2id_map[label] = cur_id
            id2lbl_map[cur_id] = label
    return lbl2id_map, id2lbl_map

In [ ]:
base_data_dir = '/content/OCR_Licplate'

train_img_dir = os.path.join(base_data_dir, 'train')
valid_img_dir = os.path.join(base_data_dir, 'val')
train_lbl_path = 'train_label.txt'
valid_lbl_path = 'valid_label.txt'
lbl2id_map_path = 'lbl2id_map.txt'

train_max_label_len = statistics_max_len_label(train_lbl_path)
valid_max_label_len = statistics_max_len_label(valid_lbl_path)
max_label_len = max(train_max_label_len, valid_max_label_len)
print(f"max len {max_label_len}")


max len 9


In [ ]:
import math
import copy
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import os
import time
import copy
import numpy as np
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings('ignore')

from torch.utils.data import Dataset

In [ ]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))


# class SimpleLossCompute:
#     "A simple loss compute and train function."
#     def __init__(self, generator, criterion, opt=None):
#         self.generator = generator
#         self.criterion = criterion
#         self.opt = opt
        
#     def __call__(self, x, y, norm):
#         x = self.generator(x)
#         x_ = x.contiguous().view(-1, x.size(-1))
#         y_ = y.contiguous().view(-1)
#         loss = self.criterion(x_, y_)
#         loss /= norm
#         loss.backward()
#         if self.opt is not None:
#             self.opt.step()
#             self.opt.zero_grad()
#         return loss.item() * norm

#simplelosscompute

In [ ]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        """
        norm: loss的归一化系数，用batch中所有有效token数即可
        """
        x = self.generator(x)
        x_ = x.contiguous().view(-1, x.size(-1))
        y_ = y.contiguous().view(-1)
        loss = self.criterion(x_, y_)
        loss /= norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        #return loss.data[0] * norm  # TODO
        return loss.item() * norm

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Encoder(nn.Module):
    """
    Encoder
    The encoder is composed of a stack of N=6 identical layers.
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# We employ a residual connection around each of the two sub-layers, followed by layer normalization
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, feature_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(feature_size))
        self.b_2 = nn.Parameter(torch.zeros(feature_size))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        sublayer_out = sublayer(x)
        sublayer_out = self.dropout(sublayer_out)
        x_norm = x + self.norm(sublayer_out)
        return x_norm


class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        # attention sub layer
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # feed forward sub layer
        z = self.sublayer[1](x, self.feed_forward)
        return z


# Decoder
# The decoder is also composed of a stack of N=6 identical layers.

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


# Attention
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


# Embeddings and Softmax
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        embedds = self.lut(x)
        return embedds * math.sqrt(self.d_model)

# Positional Encoding
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [ ]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.model_size = model_size
        self._rate = 0
    
    def state_dict(self):
        """Returns the state of the warmup scheduler as a :class:`dict`.
        It contains an entry for every variable in self.__dict__ which
        is not the optimizer.
        """
        return {key: value for key, value in self.__dict__.items() if key != 'optimizer'}
    
    def load_state_dict(self, state_dict):
        """Loads the warmup scheduler's state.
        Arguments:
            state_dict (dict): warmup scheduler state. Should be an object returned
                from a call to :meth:`state_dict`.
        """
        self.__dict__.update(state_dict) 
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

#OCR_BY_TRANSFORMER

In [ ]:
class Data_Dataset(Dataset):

    def __init__(self, dataset_root_dir, lbl2id_map, sequence_len, phase='train', pad=0):
        if phase == 'train':
            self.img_dir = os.path.join(dataset_root_dir, 'train/img/')
            self.lbl_path = 'train_label.txt'
        elif phase == 'valid':
            self.img_dir = os.path.join(dataset_root_dir, 'val/img/')
            self.lbl_path = 'valid_label.txt'
        else:
            self.img_dir = os.path.join(dataset_root_dir, 'test/img/')
            self.lbl_path = 'test_label.txt'

        self.lbl2id_map = lbl2id_map
        self.pad = pad
        self.sequence_len = sequence_len


        self.resize = transforms.Resize((32, 160))


        self.imgs_list = []
        self.lbls_list = []

        with open(self.lbl_path, 'r') as reader:
            for line in reader:
                items = line.rstrip().split(',')
                img_name = items[0]
                lbl_str = items[1]

                self.imgs_list.append(img_name)
                self.lbls_list.append(lbl_str)

        # self.color_trans = transforms.ColorJitter(0.1, 0.1, 0.1)
        self.trans_Normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.ColorJitter(0.1, 0.1, 0.1),
            transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
        ])

    def __getitem__(self, index):
        img_name = self.imgs_list[index]
        img_path = os.path.join(self.img_dir, img_name)
        lbl_str = self.lbls_list[index]

        # load image
        img = Image.open(img_path).convert('RGB')
        h_new = int(32)
        w_new = int(160)
        img_resize = img.resize((w_new, h_new), Image.BILINEAR)
        
        img_input = self.trans_Normalize(img_resize)

        # encode_mask = [1] * ratio + [0] * (self.max_ratio - ratio)
        encode_mask = [1] * int(56) + [0] * int(24) #112 + 112 (66 + 14) 56 + 24
        encode_mask = torch.tensor(encode_mask)
        encode_mask = (encode_mask != 0).unsqueeze(0)

        # ground truth label
        gt = []
        gt.append(1)
        for lbl in lbl_str:
            gt.append(self.lbl2id_map[lbl])
        gt.append(2)
        for i in range(len(lbl_str), self.sequence_len):
            gt.append(0)
        gt = gt[:self.sequence_len + 2]

        # decoder
        decode_in = gt[:-1]
        decode_in = torch.tensor(decode_in)
        # decoder
        decode_out = gt[1:]
        decode_out = torch.tensor(decode_out)
        # decoder_mask
        decode_mask = self.make_std_mask(decode_in, self.pad)
        # tokens
        ntokens = (decode_out != self.pad).data.sum()

        return img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens

    @staticmethod
    def make_std_mask(tgt, pad):

        tgt_mask = (tgt != pad)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        tgt_mask = tgt_mask.squeeze(0)  # subsequent (1, N, N)
        return tgt_mask

    def __len__(self):
        return len(self.imgs_list)

In [ ]:
def see_plot(pict, color='gray', size=(4,4), title='None'):
    plt.figure(figsize=size)
    plt.imshow(pict, cmap=color)
    plt.title(title)
    #plt.grid()
    #plt.axis('off')
    plt.show()
    
def plot_samples_on_epoch(samples, nrow=4, chanels=3, size=(16,16), title='None'):
    grid_img = torchvision.utils.make_grid(samples, nrow=nrow)
    if chanels==1:
         see_plot(grid_img.permute(1, 2, 0)*255, size=size, title=title)
    else:
         see_plot(grid_img.permute(1, 2, 0), size=size, title=title)
    #see_plot(grid_img, size=size, title=title)

In [ ]:
# # Model Architecture
# class OCR_EncoderDecoder(nn.Module):
#     """
#     A standard Encoder-Decoder architecture. 
#     Base for this and many other models.
#     """
#     def __init__(self, decoder, src_embed, src_position, tgt_embed, generator):
#         super(OCR_EncoderDecoder, self).__init__()
#         self.decoder = decoder
#         self.src_embed = src_embed    # input embedding module(input embedding + positional encode)
#         self.src_position = src_position
#         self.tgt_embed = tgt_embed    # ouput embedding module
#         self.generator = generator    # output generation module
        
#     def forward(self, src, tgt, src_mask, tgt_mask):
#         "Take in and process masked src and target sequences."
#         memory = self.encode(src, src_mask)
#         res = self.decode(memory, src_mask, tgt, tgt_mask)
#         return res
    
#     def encode(self, src, src_mask):
#         # feature extract
#         src_embedds = self.src_embed(src)
#         src_embedds = torch.flatten(src_embedds,2,3)
#         src_embedds = src_embedds.permute(0, 2, 1)

#         # position encode
#         # src_embedds = self.src_position(src_embedds)
#         return src_embedds

#         # return self.encoder(src_embedds, src_mask)
    
#     def decode(self, memory, src_mask, tgt, tgt_mask):
#         target_embedds = self.tgt_embed(tgt)
#         return self.decoder(target_embedds, memory, src_mask, tgt_mask)


# def make_ocr_model(tgt_vocab, N=3, d_model=128, d_ff=128*4, h=4, dropout=0.1):
#     c = copy.deepcopy

#     backbone = models.resnet18(pretrained=True)
#     backbone = nn.Sequential(*list(backbone.children())[:-4])

#     attn = MultiHeadedAttention(h, d_model)
#     ff = PositionwiseFeedForward(d_model, d_ff, dropout)
#     position = PositionalEncoding(d_model, dropout)

#     model = OCR_EncoderDecoder(
#         Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
#         backbone,
#         c(position),
#         nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
#         Generator(d_model, tgt_vocab))
    
#     for child in model.children():
#         if child is backbone:
#             for param in child.parameters():
#                 param.requires_grad = False
#             continue
#         for p in child.parameters():
#             if p.dim() > 1:
#                 nn.init.xavier_uniform_(p)
#     return model


In [ ]:
# Model Architecture
class OCR_EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. 
    Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, src_position, tgt_embed, generator):
        super(OCR_EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed    # input embedding module(input embedding + positional encode)
        self.src_position = src_position
        self.tgt_embed = tgt_embed    # ouput embedding module
        self.generator = generator    # output generation module
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        memory = self.encode(src, src_mask)
        res = self.decode(memory, src_mask, tgt, tgt_mask)
        return res
    
    def encode(self, src, src_mask):
        # feature extract
        src_embedds = self.src_embed(src)
        #############
        #src_embedds = src_embedds.squeeze(-2)
        src_embedds = torch.flatten(src_embedds,2,3)
        src_embedds = src_embedds.permute(0, 2, 1)

        # position encode
        src_embedds = self.src_position(src_embedds)

        return self.encoder(src_embedds, src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        target_embedds = self.tgt_embed(tgt)
        return self.decoder(target_embedds, memory, src_mask, tgt_mask)


def make_ocr_model(tgt_vocab, N=3, d_model=128, d_ff=128*4, h=4, dropout=0.1):

    c = copy.deepcopy

    backbone = models.resnet18(pretrained=True)
    backbone = nn.Sequential(*list(backbone.children())[:-4])

    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    model = OCR_EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        backbone,
        c(position),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    for child in model.children():
        if child is backbone:
            for param in child.parameters():
                param.requires_grad = False
            continue
        for p in child.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    return model

In [ ]:
def run_epoch(data_loader, model, loss_compute, device=None):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_loader):
        #if device == "cuda":
        #    batch.to_device(device) 
        img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
        img_input = img_input.to(device)
        encode_mask = encode_mask.to(device)                                
        decode_in = decode_in.to(device)                                
        decode_out = decode_out.to(device)                    
        decode_mask = decode_mask.to(device)
        ntokens = torch.sum(ntokens).to(device)

        out = model.forward(img_input, decode_in, encode_mask, decode_mask)

        loss = loss_compute(out, decode_out, ntokens)
        total_loss += loss
        total_tokens += ntokens
        tokens += ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens


# greedy decode
def greedy_decode(model, src, src_mask, max_len, start_symbol, end_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data).long()
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        next_word = torch.ones(1, 1).type_as(src.data).fill_(next_word).long()
        ys = torch.cat([ys, next_word], dim=1)

        next_word = int(next_word)
        if next_word == end_symbol:
            break
        #ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    ys = ys[0, 1:]
    return ys


# def check_correct(pred, label):
#     pred_len = pred.shape[0]
#     label = label[:pred_len]
#     is_correct = 1 if label.equal(pred) else 0
#     return is_correct

def check_correct(pred, label):
  label_non_zero = label.nonzero().shape[0]
  label = label[:label_non_zero]
  is_correct = 1 if label.equal(pred) else 0
  return is_correct

In [ ]:
device = torch.device('cuda')
nrof_epochs = 200
batch_size = 512
model_save_path = 'ex1_ocr_model.pth'
seq_len = 10

lbl2id_map, id2lbl_map = load_lbl2id_map('/content/lbl2id_map.txt')

sequence_len = max(train_max_label_len, valid_max_label_len)

#dataloader
train_dataset = Data_Dataset(base_data_dir, lbl2id_map, seq_len, 'train', pad=0) #sequence_len
valid_dataset = Data_Dataset(base_data_dir, lbl2id_map, seq_len, 'valid', pad=0)
test_dataset = Data_Dataset(base_data_dir, lbl2id_map, seq_len, 'test', pad=0)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=4)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=4)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=4)

In [ ]:
# build model
# use transformer as ocr recognize model
tgt_vocab = len(lbl2id_map.keys())
d_model = 128
ocr_model = make_ocr_model(tgt_vocab, N=3, d_model=d_model, d_ff=128*4, h=4, dropout=0.15)
ocr_model.to(device)

# train prepare
criterion = LabelSmoothing(size=tgt_vocab, padding_idx=0, smoothing=0.0)

# choose a optimizer
#optimizer = torch.optim.Adam(ocr_model.parameters(), lr=0.0005, betas=(0.9, 0.98), eps=1e-9)
optimizer = torch.optim.Adam(ocr_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
#optimizer = torch.optim.SGD(ocr_model.parameters(), lr=0.001, momentum=0.9)
model_opt = NoamOpt(d_model,200,optimizer)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
for epoch in range(nrof_epochs):
  print(f"\nepoch {epoch}")

  print("train...")
  ocr_model.train()
  loss_compute = SimpleLossCompute(ocr_model.generator, criterion, model_opt)
  #loss_compute = SimpleLossCompute(ocr_model.generator, criterion, optimizer)
  train_mean_loss = run_epoch(train_loader, ocr_model, loss_compute, device)

  if epoch % 20 == 0:
    print("valid...")
    ocr_model.eval()
    valid_loss_compute = SimpleLossCompute(ocr_model.generator, criterion, None)
    valid_mean_loss = run_epoch(valid_loader, ocr_model, valid_loss_compute, device)
    print(f"valid loss: {valid_mean_loss}")

# save model
torch.save(ocr_model.state_dict(), model_save_path)


epoch 0
train...
Epoch Step: 1 Loss: 4.508730 Tokens per Sec: 737.568726
Epoch Step: 51 Loss: 2.426189 Tokens per Sec: 6279.211426
valid...
Epoch Step: 1 Loss: 1.979805 Tokens per Sec: 2946.541748
valid loss: 1.9730536937713623

epoch 1
train...
Epoch Step: 1 Loss: 2.537446 Tokens per Sec: 2280.020996
Epoch Step: 51 Loss: 1.986830 Tokens per Sec: 6360.949219

epoch 2
train...
Epoch Step: 1 Loss: 1.843843 Tokens per Sec: 2942.832275
Epoch Step: 51 Loss: 1.425876 Tokens per Sec: 6214.872559

epoch 3
train...
Epoch Step: 1 Loss: 0.921072 Tokens per Sec: 2814.475586
Epoch Step: 51 Loss: 0.614039 Tokens per Sec: 6164.723145

epoch 4
train...
Epoch Step: 1 Loss: 0.502140 Tokens per Sec: 2663.722900
Epoch Step: 51 Loss: 0.411506 Tokens per Sec: 6109.266113

epoch 5
train...
Epoch Step: 1 Loss: 0.422446 Tokens per Sec: 1890.618164
Epoch Step: 51 Loss: 0.382465 Tokens per Sec: 6470.503418

epoch 6
train...
Epoch Step: 1 Loss: 0.340315 Tokens per Sec: 2066.650146
Epoch Step: 51 Loss: 0.303805 T

In [ ]:
# def acc_token(cur, pred):
#   #cur = cur.numpy()
#   pred = pred.numpy()
#   if len(cur) != len(pred):
#     while len(cur) != len(pred):
#       pred = np.append(pred, 0)
#     return (cur == pred).sum() / len(cur) * 100
  
#   else:
#     return (cur == pred).sum() / len(cur) * 100

In [ ]:
import time

In [ ]:
ocr_model.eval()
print("\n------------------------------------------------")
print("greedy decode trainset")
total_img_num = 0
total_correct_num = 0
start = time.time()

for batch_idx, batch in enumerate(train_loader):
  img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
  img_input = img_input.to(device)                        
  encode_mask = encode_mask.to(device)                                

  bs = img_input.shape[0]
  for i in range(bs):
      cur_img_input = img_input[i].unsqueeze(0)
      cur_encode_mask = encode_mask[i].unsqueeze(0)
      cur_decode_out = decode_out[i]
      #gredy_decode max_len = sequence_len
      pred_result = greedy_decode(ocr_model, cur_img_input, cur_encode_mask, max_len=11, start_symbol=1, end_symbol=2)
      pred_result = pred_result.cpu()
      #acc_token = acc_token(cur_decode_out, pred_result)

      is_correct = check_correct(pred_result, cur_decode_out)
      total_correct_num += is_correct
      total_img_num += 1
      # if not is_correct:
      #     print("----")
      #     print("имеющаяся последовательность", cur_decode_out)
      #     print("предсказанная последовательность", pred_result)
end = time.time()

total_correct_rate = total_correct_num / total_img_num * 100
print(f"total correct rate of trainset: {total_correct_rate}%")
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")


------------------------------------------------
greedy decode trainset
total correct rate of trainset: 98.17747357336681%
The time of execution of above program is : 3178194.90814209 ms


In [ ]:
print("\n------------------------------------------------")
print("greedy decode validset")
total_img_num = 0
total_correct_num = 0
start = time.time()
for batch_idx, batch in enumerate(valid_loader):
  img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
  img_input = img_input.to(device)                        
  encode_mask = encode_mask.to(device)                                

  bs = img_input.shape[0]
  for i in range(bs):
      cur_img_input = img_input[i].unsqueeze(0)
      cur_encode_mask = encode_mask[i].unsqueeze(0)
      cur_decode_out = decode_out[i]
      
      pred_result = greedy_decode(ocr_model, cur_img_input, cur_encode_mask, max_len=11, start_symbol=1, end_symbol=2)
      pred_result = pred_result.cpu()

      is_correct = check_correct(pred_result, cur_decode_out)
      total_correct_num += is_correct
      total_img_num += 1
      # if not is_correct:
      #     print("имеющаяся последовательность", cur_decode_out)
      #     print("предсказанная последовательность", pred_result)
end = time.time()
 
total_correct_rate = total_correct_num / total_img_num * 100
print(f"total correct rate of validset: {total_correct_rate}%")
# print the difference between start
# and end time in milli. secs
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")


------------------------------------------------
greedy decode validset
total correct rate of validset: 95.97384017984876%
The time of execution of above program is : 319013.70668411255 ms


In [ ]:
print("\n------------------------------------------------")
print("greedy decode testset")
total_img_num = 0
total_correct_num = 0
start = time.time()
for batch_idx, batch in enumerate(test_loader):
  img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
  img_input = img_input.to(device)                        
  encode_mask = encode_mask.to(device)                                

  bs = img_input.shape[0]
  for i in range(bs):
      cur_img_input = img_input[i].unsqueeze(0)
      cur_encode_mask = encode_mask[i].unsqueeze(0)
      cur_decode_out = decode_out[i]
      
      pred_result = greedy_decode(ocr_model, cur_img_input, cur_encode_mask, max_len=11, start_symbol=1, end_symbol=2)
      pred_result = pred_result.cpu()

      is_correct = check_correct(pred_result, cur_decode_out)
      total_correct_num += is_correct
      total_img_num += 1
      # if not is_correct:
      #     print("имеющаяся последовательность", cur_decode_out)
      #     print("предсказанная последовательность", pred_result)
end = time.time()

total_correct_rate = total_correct_num / total_img_num * 100
print(f"total correct rate of testset: {total_correct_rate}%")
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")


------------------------------------------------
greedy decode testset
total correct rate of testset: 94.97363796133568%
The time of execution of above program is : 185765.48719406128 ms
